#### concepts to know about
- partial
- __call__ for classes
- *args and **kwargs
- callbacks
- __dunder__ thingies

In [ ]:
import torch
import matplotlib.pyplot as plt
import random

### Callbacks

In [ ]:
import ipywidgets as widgets

In [ ]:
w = widgets.Button(description='Click me')

In [ ]:
def f(o): print('hi')

In [ ]:
w.on_click(f)

In [ ]:
w

In [ ]:
w.click()

In [ ]:
from time import sleep

In [ ]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [ ]:
slow_calculation()

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [ ]:
def show_progress(epoch):
    print(f'Awesome! Epoch: {epoch}')

In [ ]:
slow_calculation(show_progress)

In [ ]:
slow_calculation(lambda epoch: print(f'Awesome! Epoch: {epoch}'))

In [ ]:
def show_progress(exclamation, epoch):
    print(f'Awesome! {exclamation}, epoch: {epoch}')

In [ ]:
slow_calculation(lambda epoch: show_progress('Job', epoch))

In [ ]:
def f(a, b, c):
    print(a, b, c)

In [ ]:
from functools import partial

In [ ]:
f(1, 2, 3)

In [ ]:
g = partial(f, 1, 2)

In [ ]:
g(3)

In [ ]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f'Awesome job {exclamation}, epoch: {epoch}')
    return _inner

In [ ]:
slow_calculation(make_show_progress("Nice"))

In [ ]:
slow_calculation(partial(show_progress, 'Nice'))

### Callbacks as callable classes

In [ ]:
class ProgressShowingCallback():
    def __init__(self, exclamation): self.exclamation = exclamation
    def __call__(self, epoch):
        print(f'Awesome! {self.exclamation}, epoch: {epoch}')

In [ ]:
cb = ProgressShowingCallback("Nice")

In [ ]:
cb(1)

In [ ]:
slow_calculation(cb)

### Multiple callback funcs; *args and **kwargs

In [ ]:
def f(*args, **kwargs):
    # print(type(args), type(kwargs))
    print(f"args: {args}, kwargs: {kwargs}")

In [ ]:
f(3, 'a', thing1='hello')

In [ ]:
def g(a, b, c=0):
    print(a, b, c)

In [ ]:
args=[1,2]
kwargs={'c': 3}
g(*args, **kwargs)

- Passing args and kwargs
- Functions which accept these as params
- Can be used to abosrb all the unused args as well

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, res)
    
    return res

In [ ]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwarfs): print('Done')

In [ ]:
slow_calculation(PrintStepCallback())

In [ ]:
class PrintStatusCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwargs): print(f"Epoch: {args[0]}, val: {args[1]}")

In [ ]:
slow_calculation(PrintStatusCallback())

In [ ]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb,self.res = cb, 0
        
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i * i
            sleep(1)
            if self.callback('after_calc', i):
                print('early stop')
                break

In [ ]:
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f'After {epoch}: {calc.res}')
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res * 2
        

In [ ]:
c = SlowCalculator(ModifyingCallback())

In [ ]:
c.calc()

In [ ]:
c.res

In [ ]:
getattr(c, 'calc')()

In [ ]:
class SloppyAdder():
    def __init__(self, o): self.o=o
    def __add__(self, s): return SloppyAdder(self.o + s.o + 0.01)
    def __repr__(self): return str(self.o)

In [ ]:
s1 = SloppyAdder(1)
s2 = SloppyAdder(2)
s1+s2

In [ ]:
class B:
    a,b=1,2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello {k}'
    
    def abcd(self):
        return 'abcd'

In [ ]:
b = B()

In [ ]:
b.a

In [ ]:
b.foo

In [ ]:
getattr(b, 'a')

In [ ]:
getattr(b, 'abc')

In [ ]:
b.abcd()

In [ ]:
hasattr(b, 'abcd')

In [ ]:
getattr(b, 'a')

In [ ]:
getattr(b, 'b')

In [ ]:
getattr(b, 'abc')